In [1]:
import pandas as pd
import numpy as np
import magic
from matplotlib import pyplot as plt
import importlib
%matplotlib inline
importlib.reload(magic)

<module 'magic' from 'D:\\work\\icfpc2017\\python\\magic\\__init__.py'>

In [2]:
runner = magic.Fluent()
(runner
     .create_historical_players(3)
     .battles_on_map('sample.json', 2, 30)
     .battles_on_map('Sierpinski-triangle.json', 3, 40)
     .battles_on_map('gothenburg-sparse.json', 8, 60)
     .experiment('Historical')
     .run().dump().store_pointwise('historical.csv')
)
     

In [3]:
runner.token

99838

In [4]:

data = pd.read_csv('historical.csv', index_col=False)

data['tournament'] = data.tournament_scores/data.num_players
data['effectiveness'] = np.log(data.scores+1)/np.log(data.map_rivers_count/data.num_players)

In [5]:
columns  = ['tournament', 'effectiveness','scores']
groups = data.groupby(['server_name','map']).mean().loc[:,columns].reset_index()
pd.pivot_table(groups, columns='server_name', index='map', values = columns)

tournament               \
server_name              ConnectClosestMinesAi LochKillerAi   
map                                                           
Sierpinski-triangle.json              0.707317     0.740741   
gothenburg-sparse.json                0.527778     0.595092   
sample.json                           0.791667     0.650000   

                                                            effectiveness  \
server_name              MaxReachableVertexWeightAi ConnectClosestMinesAi   
map                                                                         
Sierpinski-triangle.json                   0.565891              2.126613   
gothenburg-sparse.json                     0.559066              2.017946   
sample.json                                0.812500              1.832165   

                                                                  \
server_name              LochKillerAi MaxReachableVertexWeightAi   
map                                                                
Sierpinski-triangle.json     2.159811                   2.064013   
gothenburg-sparse.json       2.104517                   2.022083   
sample.json                  1.826727                   1.818474   

                                        scores                 \
server_name              ConnectClosestMinesAi   LochKillerAi   
map                                                             
Sierpinski-triangle.json           1136.829268    1253.277778   
gothenburg-sparse.json           100238.096296  161527.613497   
sample.json                          25.666667      25.450000   

                                                     
server_name              MaxReachableVertexWeightAi  
map                                                  
Sierpinski-triangle.json                 975.534884  
gothenburg-sparse.json                 95202.565934  
sample.json                               25.062500